In [5]:
#Import dependencies
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import random_projection

#To perform Hot Encoding
from sklearn.preprocessing import StandardScaler, LabelEncoder
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense

#To trasnform data
from datetime import datetime as dt

In [6]:
df = pd.read_csv('Model_Data.csv')
df.head(5)


/Users/yuh13/anaconda3/envs/PythonData/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,IB Material - Key,Installed At - Key,RMA Create Date,Defective,Serial Number,Shipped Day,Lifespan(days)
0,0,81Y8237,100034495,3/20/2017,1,233G04B,10/12/2012,1620.0
1,1,81Y8349,101400565,1/31/2018,1,231S01X,7/18/2011,2389.0
2,2,95Y0510,101183390,4/12/2017,1,EM0711427457,10/29/2013,1261.0
3,3,95Y0510,101183390,4/27/2017,1,EM0711427505,12/13/2012,1596.0
4,4,95Y0510,101183390,4/27/2017,1,EM0711427426,12/13/2012,1596.0


In [7]:
df = df.drop(['Serial Number', 'RMA Create Date', 'Shipped Day'], axis=1)
df = df.dropna(subset=['Lifespan(days)'])
print("final dataframe shape: ", df.shape)


final dataframe shape:  (1408350, 5)


In [8]:
model_df = pd.get_dummies(df['IB Material - Key'], sparse=True)
model_df['Lifespan(days)'] = df['Lifespan(days)']
print("dummy shape: ", model_df.shape)

dummy shape:  (1408350, 2353)


In [12]:
transformer = random_projection.GaussianRandomProjection(n_components=100)


In [13]:
X_new = transformer.fit_transform(model_df)

In [15]:
y = df['Defective']


In [16]:
X_train, X_test, y_train, y_test = train_test_split(X_new, y, random_state=13)
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [17]:
training_score = model.score(X_train, y_train)
testing_score = model.score(X_test, y_test)

print(f"Training Score: {training_score}")
print(f"Testing Score: {testing_score}")

Training Score: 0.28614260318826445
Testing Score: 0.281903183188574


In [19]:
X_new.shape

(1408350, 100)

In [28]:
y.shape

(1408350,)

In [35]:

model_2 = Sequential()
number_inputs = 100
number_hidden_nodes = 4
number_classes = 2 
model_2.add(Dense(units=number_hidden_nodes, activation='relu', input_dim=number_inputs))
model_2.add(Dense(units=100, activation='relu'))
model_2.add(Dense(units=number_classes, activation='softmax'))
model_2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 4)                 404       
_________________________________________________________________
dense_12 (Dense)             (None, 100)               500       
_________________________________________________________________
dense_13 (Dense)             (None, 2)                 202       
Total params: 1,106
Trainable params: 1,106
Non-trainable params: 0
_________________________________________________________________


In [38]:
model_2.compile(optimizer='adam',
             loss = 'sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [39]:
model_2.fit(X_train,
         y_train,
         epochs=60,
         shuffle=True,
         verbose=2)

Epoch 1/60
 - 28s - loss: 0.1379 - acc: 0.9686
Epoch 2/60
 - 27s - loss: 0.1279 - acc: 0.9709
Epoch 3/60
 - 28s - loss: 0.1258 - acc: 0.9715
Epoch 4/60
 - 27s - loss: 0.1250 - acc: 0.9717
Epoch 5/60
 - 28s - loss: 0.1246 - acc: 0.9718
Epoch 6/60
 - 29s - loss: 0.1244 - acc: 0.9719
Epoch 7/60
 - 29s - loss: 0.1240 - acc: 0.9720
Epoch 8/60
 - 27s - loss: 0.1241 - acc: 0.9720
Epoch 9/60
 - 26s - loss: 0.1236 - acc: 0.9721
Epoch 10/60
 - 27s - loss: 0.1238 - acc: 0.9721
Epoch 11/60
 - 27s - loss: 0.1233 - acc: 0.9723
Epoch 12/60
 - 27s - loss: 0.1232 - acc: 0.9723
Epoch 13/60
 - 27s - loss: 0.1231 - acc: 0.9723
Epoch 14/60
 - 27s - loss: 0.1228 - acc: 0.9724
Epoch 15/60
 - 29s - loss: 0.1229 - acc: 0.9724
Epoch 16/60
 - 31s - loss: 0.1226 - acc: 0.9725
Epoch 17/60
 - 29s - loss: 0.1229 - acc: 0.9724
Epoch 18/60
 - 27s - loss: 0.1224 - acc: 0.9726
Epoch 19/60
 - 27s - loss: 0.1221 - acc: 0.9726
Epoch 20/60
 - 27s - loss: 0.1221 - acc: 0.9727
Epoch 21/60
 - 28s - loss: 0.1219 - acc: 0.9727
E

In [42]:
model_loss, model_accuracy = model_2.evaluate(
    X_test, y_test, verbose=2)
print(f"Deep Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

Deep Neural Network - Loss: 0.1196103119331254, Accuracy: 0.973316330008407


In [41]:
model_2.save("RMA_Model.h5")

In [43]:
X_new[0]

array([ 130.04839055, -179.8945108 ,  217.75512284,  164.18009332,
        -75.16615295, -168.23520278, -154.17487793,   -3.42173998,
         46.49726207, -203.70526882,  201.80740039,   70.03116477,
         10.66524273, -259.94217517,  220.16440269,  111.25939213,
        -90.47450757,  204.66502002,  171.55106624, -181.11677939,
       -114.77412872,   51.68301902,  129.25554644, -150.31873479,
        150.12681307, -319.52681038,  120.22701685, -128.49408975,
         65.10170826,  125.17995104, -215.06188076,  -72.35549374,
        353.89026557,  -79.1960382 ,  316.45716662, -162.39556551,
        206.41284192,  249.61014695,  -70.60626088,   44.91097615,
        439.69141167,  191.5919233 ,   45.9415328 , -105.83107911,
        170.50584102,   44.9142488 ,  -86.08739579, -524.39494114,
        150.77557185,  218.57275807,  261.83325552,  266.11241934,
        119.51226489, -137.14098705,  -75.58218435, -211.97750862,
       -101.45358187,   24.13362785,   98.1282133 ,  100.42616

In [45]:
model_2.predict_classes(X_new[0][0])

ValueError: Error when checking input: expected dense_11_input to have 2 dimensions, but got array with shape ()

In [ ]:
for x in range 2352:
    

In [46]:
model_df.columns


Index(['81Y8237', '81Y8349', '95Y0510', '95Y0512', 'ACX-MIC-16CHE1T1CE',
       'ACX-MIC-6GE-CU-SFP', 'ACX-MIC4OC31OC12CE', 'ACX1000-DC', 'ACX1100-AC',
       'ACX1100-DC',
       ...
       'XENPAK-1XGE-LR', 'XENPAK-1XGE-SR', 'XFP-10G-CBANDT50ZR',
       'XFP-10G-E-OC192IR2', 'XFP-10G-L-OC192SR1', 'XFP-10G-S',
       'XFP-10G-Z-OC192LR2', 'XFP-10GE-LR', 'XFP-10GE-SR', 'Lifespan(days)'],
      dtype='object', length=2353)